## Processing train gfs data

In [ ]:
import os
import sys
import requests
import pygrib
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import datetime

WORK_DIR = Path('../data/')
SAVE_DIR = Path('../data/raw/train_gfs_data/')
!mkdir {SAVE_DIR}
WORK_DIR.exists()

In [ ]:
grid_data = pd.read_csv(WORK_DIR / 'grid_metadata.csv')
grid_to_location = {g: l for g, l in zip(grid_data['grid_id'].values, grid_data['location'].values)}
grid_data.head()

In [ ]:
train_data = pd.read_csv(WORK_DIR / 'train_labels.csv')
train_data['location'] = train_data['grid_id'].apply(lambda x: grid_to_location[x])
train_data

In [ ]:
train_data = train_data[['datetime', 'location']]
print(f"Before dropping duplicates: {train_data.shape}")
train_data = train_data.drop_duplicates()
print(f"After dropping duplicates: {train_data.shape}")

In [ ]:
def check_file_status(filepath, filesize):
    # sys.stdout.write('\r')
    # sys.stdout.flush()
    size = int(os.stat(filepath).st_size)
    percent_complete = (size/filesize)*100
    # sys.stdout.write('%.3f %s' % (percent_complete, '% Completed'))
    # sys.stdout.flush()

# Write your email and password of ncar account
email = ""
pswd = ""

url = 'https://rda.ucar.edu/cgi-bin/login'
values = {'email' : email, 'passwd' : pswd, 'action' : 'login'}
# Authenticate
ret = requests.post(url,data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)
DSPATH = 'https://rda.ucar.edu/data/ds084.1/'

In [ ]:
location_map = {
    'Los Angeles (SoCAB)': 'la',
    'Taipei': 'tpe', 
    'Delhi': 'dl'
}

In [ ]:
def get_nearest_cycle(dt):
    hour = dt.hour
    ans = None
    for h in [0, 6, 12, 18]:
        if hour >= h:
            ans = h
    return ans
    
def get_nearest_forecast(dt):
    cycle = get_nearest_cycle(dt)
    available_forecasts = [0, 3, 6, 9, 12, 15, 18, 21]
    forecast = 0
    for f in available_forecasts:
        if cycle + f > dt.hour:
            break
        forecast = f
    return "{:02d}".format(cycle), "{:03d}".format(forecast)

In [ ]:
indices = [idx for idx in range(len(train_data))]
variables = ['Wind speed (gust)', 'Haines Index', 'Surface pressure', 'Orography', 'Temperature', 'Water equivalent of accumulated snow depth (deprecated)', 'Snow depth', 'Potential evaporation rate', 'Percent frozen precipitation', 'Convective precipitation rate', 'Precipitation rate', 'Total Precipitation', 'Convective precipitation (water)', 'Water runoff', 'Categorical snow', 'Categorical ice pellets', 'Categorical freezing rain', 'Categorical rain', 'Latent heat net flux', 'Sensible heat net flux', 'Ground heat flux', 'Momentum flux, u component', 'Momentum flux, v component', 'Zonal flux of gravity wave stress', 'Meridional flux of gravity wave stress', 'Wilting Point', 'Field Capacity', 'Sunshine Duration', 'Surface lifted index', 'Convective available potential energy', 'Convective inhibition', 'Downward short-wave radiation flux', 'Downward long-wave radiation flux', 'Upward short-wave radiation flux', 'Upward long-wave radiation flux', 'Best (4-layer) lifted index', 'Planetary boundary layer height', 'Land-sea mask', 'Sea ice area fraction', 'Albedo', 'Land-sea coverage (nearest neighbor) [land=1,sea=0]']

for idx in tqdm(indices):
    el = train_data.iloc[idx]
    dt = datetime.datetime.strptime(el['datetime'], '%Y-%m-%dT%H:%M:%SZ')
    loc = location_map[el['location']]

    cycle, forecast = get_nearest_forecast(dt)
    year = dt.year
    month = "{:02d}".format(dt.month)
    day = "{:02d}".format(dt.day)
    filename = f'{year}/{year}{month}{day}/gfs.0p25.{year}{month}{day}{cycle}.f{forecast}.grib2'
    # print(f"{dt} => {filename}")

    filename = DSPATH + filename
    file_base = os.path.basename(filename)

    req = requests.get(filename, cookies = ret.cookies, allow_redirects=True, stream=True)
    filesize = int(req.headers['Content-length'])
    with open(file_base, 'wb') as outfile:
        chunk_size=1048576
        for chunk in req.iter_content(chunk_size=chunk_size):
            outfile.write(chunk)
            if chunk_size < filesize:
                check_file_status(file_base, filesize)
    check_file_status(file_base, filesize)

    gr = pygrib.open(file_base)
    assets = {}

    for g in gr:
        if g.name in variables and g.typeOfLevel == 'surface':
            assets[g.name] = g.values

    assets['latitude'] = g.latlons()[0]
    assets['longitude'] = g.latlons()[1]

    np.savez_compressed(SAVE_DIR / f"{el['datetime']}_{loc}", **assets)
    !rm {file_base}

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from shapely.geometry import Point, Polygon
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path
import datetime
import warnings
warnings.simplefilter("ignore")

DATA_DIR = Path('../data/raw/train_gfs_data/')
assert DATA_DIR.exists(), "{} does not exist...".format(DATA_DIR)
REQUIRED_BANDS = [
    'Wind speed (gust)', 'Haines Index', 'Surface pressure', 
    'Orography', 'Temperature', 'Water equivalent of accumulated snow depth (deprecated)', 
    'Snow depth', 'Percent frozen precipitation', 'Wilting Point', 'Field Capacity',
    'Sunshine Duration', 'Surface lifted index', 'Convective available potential energy',
    'Convective inhibition', 'Best (4-layer) lifted index', 'Planetary boundary layer height',
    'Land-sea mask', 'Sea ice area fraction', 'Land-sea coverage (nearest neighbor) [land=1,sea=0]',
    'latitude', 'longitude'
]

def floor(x, n=0):
    return np.floor(x * 10**n) / 10**n

def ceil(x, n=0):
    return np.ceil(x * 10**n) / 10**n

def get_bounds(geometry):
    geometry = np.array(geometry)
    long = [
        floor(geometry[:, 0].min(), 1),
        ceil(geometry[:, 0].max(), 1)
    ]
    lat = [
        floor(geometry[:, 1].min(), 1),
        ceil(geometry[:, 1].max(), 1)
    ]
    return long, lat

def round_off(point, res):
    spread = np.arange(np.floor(point), np.ceil(point) + 1, res)
    adiff = np.abs(spread - point)
    return spread[np.argmin(adiff)]

def get_boundary(geometry, res):
    long = np.array(geometry)[:, 0]
    lat = np.array(geometry)[:, 1]
    
    min_lat, max_lat = lat.min(), lat.max()
    min_long, max_long = long.min(), long.max()
    
    min_lat = round_off(min_lat - res / 2, res)
    max_lat = round_off(max_lat + res / 2, res)
    min_long = round_off(min_long - res / 2, res)
    max_long = round_off(max_long + res / 2, res)
    
    return [min_long, max_long], [min_lat, max_lat]

def mask(filename, geometry):
    data = np.load(filename)
    assets = {}
    for key in data.keys():
        assets[key] = data[key].ravel()
    # longb, latb = get_bounds(geometry)
    longb, latb = get_boundary(geometry, 0.25)
    latitude = assets['latitude']
    longitude = assets['longitude']
    indices = (latitude >= latb[0]) & (latitude <= latb[1]) & (longitude >= longb[0]) & (longitude <= longb[1])

    new_ass = {}
    for k in assets.keys():
        new_ass[k] = assets[k][indices]
                                                                   
    df = pd.DataFrame(new_ass)
    return df

train_data = pd.read_csv("../data/train_labels.csv")
grid_data = pd.read_csv("../data/grid_metadata.csv")
location_map = {
    'Los Angeles (SoCAB)': 'la',
    'Taipei': 'tpe', 
    'Delhi': 'dl'
}

indices = [idx for idx in range(len(train_data))]
total_data = defaultdict(lambda: [])

for idx in tqdm(indices):
    el = train_data.iloc[idx]
    grid_id = el['grid_id']
    dt = el['datetime']

    grid_el = grid_data.loc[grid_data['grid_id'] == grid_id]
    loc = location_map[grid_el['location'].iloc[0]]
    
    geometry = grid_el['wkt'].values[0]
    geometry = geometry.replace('(', '', -1)
    geometry = geometry.replace(')', '', -1)
    geometry = geometry.replace(',', '', -1)
    geometry = list(map(float, geometry.split()[1:]))
    geometry = [geometry[i:i+2] for i in range(0, len(geometry), 2)]

    filename = '{}_{}.npz'.format(dt, loc)
    filename = filename.replace(':', '_', -1)
    filename = DATA_DIR / filename
    try:
        cur_data = mask(filename, geometry)
    except Exception as e:
        cur_data = pd.DataFrame({})
        print(f"{idx}: {e}")

    for key in REQUIRED_BANDS:
        try:
            _band = cur_data[key].values
            _band = np.concatenate((
                _band[_band <= 0], _band[_band > 0]
            ))
        except KeyError:
            _band = np.array([np.nan, np.nan])
        total_data[f"{key}_mean"].append(_band.mean())
        total_data[f"{key}_var"].append(_band.std() ** 2)

total_data = pd.DataFrame(total_data)
total_data.to_csv('../data/proc/train_gfs.csv', index=False)

## Processing test gfs data

In [ ]:
import os
import sys
import requests
import pygrib
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import datetime

WORK_DIR = Path('../data/')
SAVE_DIR = Path('../data/raw/test_gfs_data/')
!mkdir {SAVE_DIR}
WORK_DIR.exists()

In [ ]:
grid_data = pd.read_csv(WORK_DIR / 'grid_metadata.csv')
grid_to_location = {g: l for g, l in zip(grid_data['grid_id'].values, grid_data['location'].values)}
grid_data.head()

In [ ]:
train_data = pd.read_csv(WORK_DIR / 'submission_format.csv')
train_data['location'] = train_data['grid_id'].apply(lambda x: grid_to_location[x])
train_data

In [ ]:
train_data = train_data[['datetime', 'location']]
print(f"Before dropping duplicates: {train_data.shape}")
train_data = train_data.drop_duplicates()
print(f"After dropping duplicates: {train_data.shape}")

In [ ]:
def check_file_status(filepath, filesize):
    # sys.stdout.write('\r')
    # sys.stdout.flush()
    size = int(os.stat(filepath).st_size)
    percent_complete = (size/filesize)*100
    # sys.stdout.write('%.3f %s' % (percent_complete, '% Completed'))
    # sys.stdout.flush()

# Write your email and password of ncar account
email = ""
pswd = ""

url = 'https://rda.ucar.edu/cgi-bin/login'
values = {'email' : email, 'passwd' : pswd, 'action' : 'login'}
# Authenticate
ret = requests.post(url,data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)
DSPATH = 'https://rda.ucar.edu/data/ds084.1/'

In [ ]:
location_map = {
    'Los Angeles (SoCAB)': 'la',
    'Taipei': 'tpe', 
    'Delhi': 'dl'
}

In [ ]:
def get_nearest_cycle(dt):
    hour = dt.hour
    ans = None
    for h in [0, 6, 12, 18]:
        if hour >= h:
            ans = h
    return ans
    
def get_nearest_forecast(dt):
    cycle = get_nearest_cycle(dt)
    available_forecasts = [0, 3, 6, 9, 12, 15, 18, 21]
    forecast = 0
    for f in available_forecasts:
        if cycle + f > dt.hour:
            break
        forecast = f
    return "{:02d}".format(cycle), "{:03d}".format(forecast)

In [ ]:
indices = [idx for idx in range(len(train_data))]
variables = ['Wind speed (gust)', 'Haines Index', 'Surface pressure', 'Orography', 'Temperature', 'Water equivalent of accumulated snow depth (deprecated)', 'Snow depth', 'Potential evaporation rate', 'Percent frozen precipitation', 'Convective precipitation rate', 'Precipitation rate', 'Total Precipitation', 'Convective precipitation (water)', 'Water runoff', 'Categorical snow', 'Categorical ice pellets', 'Categorical freezing rain', 'Categorical rain', 'Latent heat net flux', 'Sensible heat net flux', 'Ground heat flux', 'Momentum flux, u component', 'Momentum flux, v component', 'Zonal flux of gravity wave stress', 'Meridional flux of gravity wave stress', 'Wilting Point', 'Field Capacity', 'Sunshine Duration', 'Surface lifted index', 'Convective available potential energy', 'Convective inhibition', 'Downward short-wave radiation flux', 'Downward long-wave radiation flux', 'Upward short-wave radiation flux', 'Upward long-wave radiation flux', 'Best (4-layer) lifted index', 'Planetary boundary layer height', 'Land-sea mask', 'Sea ice area fraction', 'Albedo', 'Land-sea coverage (nearest neighbor) [land=1,sea=0]']

for idx in tqdm(indices):
    el = train_data.iloc[idx]
    dt = datetime.datetime.strptime(el['datetime'], '%Y-%m-%dT%H:%M:%SZ')
    loc = location_map[el['location']]

    cycle, forecast = get_nearest_forecast(dt)
    year = dt.year
    month = "{:02d}".format(dt.month)
    day = "{:02d}".format(dt.day)
    filename = f'{year}/{year}{month}{day}/gfs.0p25.{year}{month}{day}{cycle}.f{forecast}.grib2'
    # print(f"{dt} => {filename}")

    filename = DSPATH + filename
    file_base = os.path.basename(filename)

    req = requests.get(filename, cookies = ret.cookies, allow_redirects=True, stream=True)
    filesize = int(req.headers['Content-length'])
    with open(file_base, 'wb') as outfile:
        chunk_size=1048576
        for chunk in req.iter_content(chunk_size=chunk_size):
            outfile.write(chunk)
            if chunk_size < filesize:
                check_file_status(file_base, filesize)
    check_file_status(file_base, filesize)

    gr = pygrib.open(file_base)
    assets = {}

    for g in gr:
        if g.name in variables and g.typeOfLevel == 'surface':
            assets[g.name] = g.values

    assets['latitude'] = g.latlons()[0]
    assets['longitude'] = g.latlons()[1]

    np.savez_compressed(SAVE_DIR / f"{el['datetime']}_{loc}", **assets)
    !rm {file_base}

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from shapely.geometry import Point, Polygon
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path
import datetime
import warnings
warnings.simplefilter("ignore")

DATA_DIR = Path('../data/raw/test_gfs_data/')
assert DATA_DIR.exists(), "{} does not exist...".format(DATA_DIR)
REQUIRED_BANDS = [
    'Wind speed (gust)', 'Haines Index', 'Surface pressure', 
    'Orography', 'Temperature', 'Water equivalent of accumulated snow depth (deprecated)', 
    'Snow depth', 'Percent frozen precipitation', 'Wilting Point', 'Field Capacity',
    'Sunshine Duration', 'Surface lifted index', 'Convective available potential energy',
    'Convective inhibition', 'Best (4-layer) lifted index', 'Planetary boundary layer height',
    'Land-sea mask', 'Sea ice area fraction', 'Land-sea coverage (nearest neighbor) [land=1,sea=0]',
    'latitude', 'longitude'
]

def floor(x, n=0):
    return np.floor(x * 10**n) / 10**n

def ceil(x, n=0):
    return np.ceil(x * 10**n) / 10**n

def get_bounds(geometry):
    geometry = np.array(geometry)
    long = [
        floor(geometry[:, 0].min(), 1),
        ceil(geometry[:, 0].max(), 1)
    ]
    lat = [
        floor(geometry[:, 1].min(), 1),
        ceil(geometry[:, 1].max(), 1)
    ]
    return long, lat

def round_off(point, res):
    spread = np.arange(np.floor(point), np.ceil(point) + 1, res)
    adiff = np.abs(spread - point)
    return spread[np.argmin(adiff)]

def get_boundary(geometry, res):
    long = np.array(geometry)[:, 0]
    lat = np.array(geometry)[:, 1]
    
    min_lat, max_lat = lat.min(), lat.max()
    min_long, max_long = long.min(), long.max()
    
    min_lat = round_off(min_lat - res / 2, res)
    max_lat = round_off(max_lat + res / 2, res)
    min_long = round_off(min_long - res / 2, res)
    max_long = round_off(max_long + res / 2, res)
    
    return [min_long, max_long], [min_lat, max_lat]

def mask(filename, geometry):
    data = np.load(filename)
    assets = {}
    for key in data.keys():
        assets[key] = data[key].ravel()
    # longb, latb = get_bounds(geometry)
    longb, latb = get_boundary(geometry, 0.25)
    latitude = assets['latitude']
    longitude = assets['longitude']
    indices = (latitude >= latb[0]) & (latitude <= latb[1]) & (longitude >= longb[0]) & (longitude <= longb[1])

    new_ass = {}
    for k in assets.keys():
        new_ass[k] = assets[k][indices]
                                                                   
    df = pd.DataFrame(new_ass)
    return df

train_data = pd.read_csv("../data/submission_format.csv")
grid_data = pd.read_csv("../data/grid_metadata.csv")
location_map = {
    'Los Angeles (SoCAB)': 'la',
    'Taipei': 'tpe', 
    'Delhi': 'dl'
}

indices = [idx for idx in range(len(train_data))]
total_data = defaultdict(lambda: [])

for idx in tqdm(indices):
    el = train_data.iloc[idx]
    grid_id = el['grid_id']
    dt = el['datetime']

    grid_el = grid_data.loc[grid_data['grid_id'] == grid_id]
    loc = location_map[grid_el['location'].iloc[0]]
    
    geometry = grid_el['wkt'].values[0]
    geometry = geometry.replace('(', '', -1)
    geometry = geometry.replace(')', '', -1)
    geometry = geometry.replace(',', '', -1)
    geometry = list(map(float, geometry.split()[1:]))
    geometry = [geometry[i:i+2] for i in range(0, len(geometry), 2)]

    filename = '{}_{}.npz'.format(dt, loc)
    filename = filename.replace(':', '_', -1)
    filename = DATA_DIR / filename
    try:
        cur_data = mask(filename, geometry)
    except Exception as e:
        cur_data = pd.DataFrame({})
        print(f"{idx}: {e}")

    for key in REQUIRED_BANDS:
        try:
            _band = cur_data[key].values
            _band = np.concatenate((
                _band[_band <= 0], _band[_band > 0]
            ))
        except KeyError:
            _band = np.array([np.nan, np.nan])
        total_data[f"{key}_mean"].append(_band.mean())
        total_data[f"{key}_var"].append(_band.std() ** 2)

total_data = pd.DataFrame(total_data)
total_data.to_csv('../data/proc/test_gfs.csv', index=False)